<a href="https://colab.research.google.com/github/jonathanjalles/pandas-zero/blob/master/agrupamento_k_means_g_means.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agrupamento baseado em centróides: k-means e g-means



Agrupamento ou *clustering*, no contexto de Machine Learning (ML), consiste em agrupar (em clusters) conjuntos de dados que possuem similaridade entre si. Nesse caso, a similaridade é medida com base na força do relacionamento entre dois pontos de dados. Algoritmos para clustering são usados principalmente em mineração de dados exploratória, já que permitem agrupar dados para descoberta de padrões, por exemplo.

Nesse notebook serão abordadas duas ***técnicas de clustering baseadas em centróides*** (k-means e g-means). Um centróide pode ser entendido como um dado (real ou imaginário) localizado no centro do cluster, a fim de representá-lo. Nessa técnica, a similaridade é medida com base na distância entre um dado e o centróide do cluster. Na figura abaixo, temos 2 clusters e seus centróides destacados pelas setas pretas.


![Exemplo de clustering](https://cdn.app.compendium.com/uploads/user/e7c690e8-6ff9-102a-ac6d-e4aebca50425/f0499405-1197-4b43-b7c5-40548eeb9f34/File/9b5d5bd03696c340d213ba9af955b13c/clustering_and_k_means_machine_learning.png)

Fonte: [K-Means Clustering in Machine Learning, Simplified](https://blogs.oracle.com/bigdata/k-means-clustering-machine-learning)

* **k-means:** De forma resumida, nesse método o próprio analista define o número de clusters (k) que serão utilizados para o agrupamento. Em seguida, são escolhidos pontos para cada centróide e cada ponto de dado tem sua distância ao centróide determinada. Com os clusters definidos são feitas iterações para calcular um novo centróide e refinar o cluster, até que não ocorra mudança no agrupamento.


![alt text](https://sandipanweb.files.wordpress.com/2017/03/kmeans1.gif?w=676)

Fonte: https://sandipanweb.wordpress.com/page/8/




* **g-means:** Esse método tenta prever qual a melhor quantidade de clusters para agrupar os dados (ou seja, tenta encontrar k), assumindo a hipótese de que cada cluster possui uma distribuição Gaussiana. Assim, o algoritmo aumenta o valor de k até que o teste aceite a hipótese como verdadeira.



Como exemplo, vamos clusterizar dados de passageiros do navio Titanic, nos grupos ***sobrevivente*** e ***não sobrevivente***. Esse exemplo é baseado no [post](www.datacamp.com/community/tutorials/k-means-clustering-python) de Sayak Paul, na comunidade do DataCamp. Aqui, vamos tentar prever se um passageiro sobreviveu ou não considerando três atributos:
* Idade
* Preço do bilhete para embarcar
* Sexo

Abaixo vamos importar os pacotes necessários para realizar a clusterização. O tratamento feito com os dados utilizando pandas não será explicado em detalhes, por não ser esse o escopo do exemplo. Nosso objetivo será obter uma função para prever se um passageiro sobreviveu ou não ao naufrágio.

In [0]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


# 1. Importando e tratando dados

Nosso conjunto de dados está disponível [aqui](https://www.kaggle.com/c/titanic).

In [0]:
dados_treino = pd.read_csv('http://s3.amazonaws.com/assets.datacamp.com/course/Kaggle/train.csv')

In [0]:
dados_treino.head()

Vamos obter algumas informações do conjunto de dados:

In [0]:
dados_treino.info()

vamos selecionar somente os atributos de interesse e alterar os nomes das colunas

In [0]:
dados_treino = dados_treino[['Survived','Fare', 'Sex','Age']]
dados_treino.columns = ['sobreviveu', 'precoBilhete', 'Sexo','Idade']
dados_treino.head()

Aqui, o valor 0 no atributo *sobreviveu* indica que o passageiro morreu no naufrágio. Vamos verificar se existem dados nulos nas colunas. O k-means não aceita valores faltando ou valores que não sejam números (trataremos isso em breve).

In [0]:
dados_treino.isna().sum()

Temos 177 linhas que não possuem o atributo Idade. Há várias formas de abordar dados faltando. Aqui, vamos preencher esses campos com a média das idades existentes. Não vamos apagar os dados para não correr o risco de ficarmos com uma amostra pequena demais para treinar o modelo.

In [0]:
dados_treino.fillna(dados_treino.mean(), inplace=True)

In [0]:
dados_treino.isna().sum()

In [0]:
dados_treino.info()

Agora temos todos as linhas do nosso conjunto de dados de treino preenchidas. Porém, o atributo *Sexo* está no formato de texto, o que não é aceito pelo algoritmo k-means. Para resolver isso, vamor fazer um feature scaling nesse atributo e aplicar o OneHotEncoding (veja esse notebook sobre transformação de dados para entender melhor).

In [0]:
labelEncoder = LabelEncoder()
labelEncoder.fit(dados_treino['Sexo'])
dados_treino['Sexo'] = labelEncoder.transform(dados_treino['Sexo'])
onehot_encoder = OneHotEncoder(sparse=False, categories='auto')
dados_sexo = np.array(dados_treino['Sexo']).reshape(len(dados_treino['Sexo']),1)
dados_treino['Sexo'] = onehot_encoder.fit_transform(dados_sexo)

In [0]:
dados_treino.head()

Temos agora um novo dataframe, onde na coluna *Sexo* o valor 0.0 representa masculino, e 1.0 representa feminino.

In [0]:
dados_treino.info()

Antes de treinar o modelo, vamos plotar um gráfico de dispersão para tentar identificar possíveis clusters. Os dados serão agrupados de acordo com o atributo *sobreviveu*.

In [0]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
x=dados_treino['precoBilhete']
y=dados_treino['Sexo']
z=dados_treino['Idade']
ax.scatter(x,y,z, c=dados_treino['sobreviveu'], s=150, label='Não sobrevivente')
ax.view_init(30, 200)
ax.set_xlabel('preço do bilhete')
#ax.set_ylabel('sexo')
ax.set_zlabel('idade')
ax.legend(dados_treino['sobreviveu'], loc='upper right')
ax.text2D(0.05, 0.95, 'Clusters', transform=ax.transAxes)
plt.show()

Uma vez que os pontos escuros representam não sobreviventes ao naufrágio, podemos concluir com base nos dados fornecidos que mais mulheres do que homens sobreviveram à tragédia. Como exite um atributo binário nos dados (sexo), temos uma separação clara nesse eixo.

# 2. Agrupando com k-means

Depois de realizar o pré processamento dos dados, vamos iniciar o treinamento do modelo. Primeiro, vamos gerar um array do numpy com os dados de treino, retirando a coluna *'Sobreviveu'*. Esse array será nosso *X*.

In [0]:
X = np.array(dados_treino.drop(['sobreviveu'], 1).astype(float))

Imprimindo as primeiras 4 linhas do array:



In [0]:
X[0:4]

Agora vamos criar os clusters. Para essa aplicação, vamos separar os dados em 4 grupos (clusters). Logo, nosso k vale 4.

In [0]:
kmeans = KMeans(n_clusters=4).fit(X)

Os clusters foram representados pelos seguintes centroides:

In [0]:
centroides = np.array(kmeans.cluster_centers_)
centroides

In [0]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(centroides[:,0],centroides[:,1],centroides[:,2], c='red', marker='x', s=150)
ax.view_init(30, 200)
ax.set_xlim(0, 600)
ax.set_ylim(0, 1)
ax.set_zlim(0, 80)
ax.set_xlabel('preço do bilhete')
ax.set_ylabel('sexo')
ax.set_zlabel('idade')
plt.show()

Se utilizarmos o modelo para clusterizar os dados, obteremos os seguintes grupos (as cores representam os clusters gerados pelo k-means):

In [0]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

x=dados_treino['precoBilhete']
y=dados_treino['Sexo']
z=dados_treino['Idade']
ax.scatter(x,y,z, marker='o', c=kmeans.labels_, s=150)
ax.view_init(30, 200)
ax.set_xlim(0, 600)
ax.set_ylim(0, 1)
ax.set_zlim(0, 80)
ax.set_xlabel('preço do bilhete')
ax.set_ylabel('sexo')
ax.set_zlabel('idade')

plt.show()

Aqui, temos uma divisão clara dos clusters em relação ao eixo "preço do bilhete". Isso ocorre por que a dimensão dos dados nesse eixo está numa ordem muito maior que os demais eixos. Para resolver esse problema, precisamos fazer uma nova transformação nos dados.

Para deixar os dados dentro da mesma escala de valores (nesse caso, entre 0 e 1), vamos aplicar o ***MinMaxScaler()*** nos dados.

In [0]:
scaler = MinMaxScaler()
x_com_scaling = scaler.fit_transform(X)
x_com_scaling

Agora vamos criar novamente os clusters, dessa vez com o nome ***kmeans_scaled***.

In [0]:
kmeans_scaled = KMeans(n_clusters=4).fit(x_com_scaling)

Os novos centróides são:

In [0]:
centroides_scaled = kmeans_scaled.cluster_centers_
centroides_scaled

Abaixo vamos plotar os novos clusters, para o novo array de dados, *x_com_scaling* (em cada eixo, o valor 1 representa o máximo do atributo nos dados originais, e o valor 0, o mínimo):

In [0]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
x=x_com_scaling[:,0]
y=x_com_scaling[:,1]
z=x_com_scaling[:,2]
ax.scatter(x,y,z, marker='o', c=kmeans_scaled.labels_, s=150)
ax.set_xlabel('preço do bilhete')
ax.set_ylabel('sexo')
ax.set_zlabel('idade')
ax.scatter(centroides_scaled[:,0], centroides_scaled[:,1], centroides_scaled[:,2], c='red', marker='x', s=300)
ax.view_init(30, 200)
plt.show()

# 3. Agrupando com g-means

Conforme citado no inicio desse notebook, nem sempre será trivial determinar o número correto de clusters k para agrupar um conjunto de dados. O algoritmo gmeans foi criado a fim de permitir a determinação adequada do valor de k, baseando-se num teste estatístico de hipótese, de que cada subconjunto (cluster) segue uma distribuição normal (Gaussiana). Por isso o método incrementa k em cada iteração a fim de determinar seu valor ótimo. Para maiores detalhes sobre o funcionamento do método, basta checar esse [artigo](papers.nips.cc/paper/2526-learning-the-k-in-k-means.pdf).

Para implementar o modelo, vamos utilizar a biblioteca [pyclustering](https://pyclustering.github.io/docs/0.9.0/html/index.html).

primeiro, vamos instalar o pacote:

In [0]:
!pip install pyclustering

vamos importar as funções **cluster_visualizer()** e **gmeans()**.
Para maiores detalhes sobre o gmeans com pyclustering, veja a [documentação](https://codedocs.xyz/annoviko/pyclustering/gmeans_8py_source.html).

In [0]:
from pyclustering.cluster import cluster_visualizer
from pyclustering.cluster.gmeans import gmeans

a variável modelo_gmeans terá o modelo treinado com os dados preprocessados e armazenados no array numpy **x_com_scaling**, com 10 iterações:

In [0]:
modelo_gmeans = gmeans(x_com_scaling).process()

In [0]:
modelo_gmeans

In [0]:
clusters = modelo_gmeans.get_clusters()
clusters

In [0]:
centroides_gmeans = gmeans_algo.get_centers()

Vejamos a quantidade de centróides determinados pelo gmeans:


In [0]:
len(centroides_gmeans)

Abaixo vamos visualizar os clusters criados pelo gmeans

In [0]:
grafico = cluster_visualizer()
grafico.append_clusters(clusters, x_com_scaling)
grafico.show(invisible_axis=False)